In [1]:
import sqlite3.dbapi2 as sqlite

In [2]:
conn = sqlite.connect('adresy.sqlite')
curs = conn.cursor()

In [3]:
curs.execute("CREATE TABLE IF NOT EXISTS obce(id_obce INTEGER PRIMARY KEY,nazov_obce TEXT);")
curs.execute("CREATE TABLE IF NOT EXISTS ulice(id_ulice INTEGER PRIMARY KEY,nazov_ulice TEXT,id_obce INTEGER,foreign key(id_obce) references obce(id_obce));")
curs.execute("CREATE TABLE IF NOT EXISTS cisla(id_domu INTEGER PRIMARY KEY,cislo_domu TEXT,latitude INTEGER, longtitude INTEGER, id_ulice INTEGER,foreign key(id_ulice) references obce(id_ulice));")

In [4]:
with open('obce.csv') as f:
    for riadok in f:
        oid,omeno=riadok[:-1].split(',')
        try:
            curs.execute("INSERT INTO obce VALUES (?,?)",(int(oid),omeno))
        except sqlite.IntegrityError:
            pass

In [5]:
with open('ulice.csv') as f:
    for riadok in f:
        uid,umeno,oid=riadok[:-1].split(',')
        try:
            curs.execute("INSERT INTO ulice VALUES (?,?,?)",(int(uid),umeno,int(oid)))
        except sqlite.IntegrityError:
            pass

In [6]:
with open('cisla.csv') as f:
    for riadok in f:
        did,cd,lat,lon,uid=riadok[:-1].split(',')
        try:
            curs.execute("INSERT INTO cisla VALUES (?,?,?,?,?)",(int(did),cd,lat,lon,int(uid)))
        except sqlite.IntegrityError:
            pass

In [ ]:
#zoznam ulic v Ziline
curs.execute("SELECT u.nazov_ulice FROM ulice u,obce o WHERE o.nazov_obce = 'Žilina' AND u.id_obce = o.id_obce")
u = curs.fetchall()
u[:10]

In [ ]:
#Pocet vsetkych ulic v Bratislave
curs.execute("SELECT COUNT(u.nazov_ulice) FROM ulice u,obce o WHERE o.nazov_obce LIKE '%Bratislava%' AND u.id_obce = o.id_obce")
curs.fetchall()

In [ ]:
#Pocet vsetkych cisel na ulici Univerzitna v Ziline
curs.execute("SELECT c.cislo_domu,latitude,longtitude FROM cisla c,ulice u,obce o WHERE  o.nazov_obce LIKE '%Žilina%'\
AND o.id_obce = u.id_obce AND u.nazov_ulice LIKE '%Univerzitná%' AND u.id_ulice = c.id_ulice GROUP BY cislo_domu")
curs.fetchall()

In [7]:
def lokalizacia_ulice(ulica,mesto):
    curs.execute("SELECT c.cislo_domu,latitude,longtitude FROM cisla c,ulice u,obce o WHERE  o.nazov_obce LIKE '%"+mesto+"%'\
    AND o.id_obce = u.id_obce AND u.nazov_ulice LIKE '%"+ulica+"%' AND u.id_ulice = c.id_ulice GROUP BY cislo_domu")
    return curs.fetchall()

In [ ]:
zoznam = lokalizacia_ulice('Univerzitná','Žilina')

In [ ]:
print(zoznam)

In [ ]:
print(lokalizacia_ulice('Farského','Bratislava'))

In [8]:
def lokalizacia_adresy(ulica,cislo,mesto):
    curs.execute("SELECT c.cislo_domu,latitude,longtitude FROM cisla c,ulice u,obce o WHERE  o.nazov_obce LIKE '%"+mesto+"%'\
    AND o.id_obce = u.id_obce AND u.nazov_ulice LIKE '%"+ulica+"%' AND u.id_ulice = c.id_ulice AND c.cislo_domu LIKE '%"+cislo+"%'")
    return curs.fetchall()

In [ ]:
print(lokalizacia_adresy('Farského','12','Bratislava'))

In [10]:
r = lokalizacia_adresy('Univerzitná','1','Žilina')
r = lokalizacia_ulice('Hlavná','Prešov')

In [12]:
import folium
from numpy import mean
C,Lat,Lon = zip(*r)
sur  = zip(Lat,Lon)
Lats,Lons=mean(Lat),mean(Lon)
map = folium.Map(location=(Lats,Lons),zoom_start=17)
for p,c in zip(sur,C):
    map.add_child(folium.CircleMarker(p,radius=6,color='red',fill_color = 'red',fill_opacity=0.3,popup="%s %s" %('Univerzitná',c)))
map.sa
map

SyntaxError: invalid syntax (3327333369.py, line 9)